# SSD300 Pascal VOC Evaluation
### Reference: https://github.com/pierluigiferrari/ssd_keras/blob/master/ssd300_evaluation_Pascal_VOC.ipynb
- Evaluate a trained SSD300 on Pascal VOC2007 test dataset using the official Matlab evaluation script that comes with the [VOCdevkit](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html).

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from keras_ssd300 import ssd_300
from keras_ssd_loss import SSDLoss
from keras_layer_AnchorBoxes import AnchorBoxes
from keras_layer_L2Normalization import L2Normalization
from ssd_box_encode_decode_utils import SSDBoxEncoder
from ssd_batch_generator import BatchGenerator
from pascal_voc_utils import predict_all_to_txt

%matplotlib inline

In [ ]:
# Set the input image size for the model.
img_height = 300
img_width = 300

# 1. Load a trained SSD

In [ ]:
model_path = 'model/ssd300_pascal_2012.h5'

# We need to create an SSDLoss object in order to pass that to the model loader.
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

K.clear_session() # Clear previous models from memory.

model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                               'L2Normalization': L2Normalization,
                                               'compute_loss': ssd_loss.compute_loss})

# 2. Create a data generator for the evaluation dataset

In [ ]:

dataset = BatchGenerator(box_output_format=['class_id', 'xmin', 'ymin', 'xmax', 'ymax'])

# TODO: Set the paths to the dataset here.
VOC_2012_images_dir      = '/datasets/pascal_voc/VOCdevkit/VOC2012/JPEGImages/'
VOC_2012_annotations_dir      = '/datasets/pascal_voc/VOCdevkit/VOC2012/Annotations/'
VOC_2012_val_image_set_filename      = '/datasets/pascal_voc/VOCdevkit/VOC2012/ImageSets/Main/val.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat',
           'chair', 'cow', 'diningtable', 'dog',
           'horse', 'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']

dataset.parse_xml(images_dirs=[VOC_2012_images_dir],
                  image_set_filenames=[VOC_2012_val_image_set_filename],
                  annotations_dirs=[VOC_2012_annotations_dir],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)

# 3. Run the predictions over the evaluation dataset

In [ ]:
# TODO: Set the batch size.
batch_size = 8

In [ ]:
predict_all_to_txt(model=model,
                   img_height=img_height,
                   img_width=img_width,
                   batch_generator=dataset,
                   batch_size=batch_size,
                   batch_generator_mode='resize',
                   classes=['background',
                            'aeroplane', 'bicycle', 'bird', 'boat',
                            'bottle', 'bus', 'car', 'cat',
                            'chair', 'cow', 'diningtable', 'dog',
                            'horse', 'motorbike', 'person', 'pottedplant',
                            'sheep', 'sofa', 'train', 'tvmonitor'],
                   out_file_prefix='comp3_det_test_',
                   confidence_thresh=0.01,
                   iou_threshold=0.45,
                   top_k=200,
                   pred_coords='centroids',
                   normalize_coords=True)

# 4. Run the evaluation
Now that we've generated the results text files for each class in the right format, we still have to evaluate the results.

This repository doesn't provide its own Pascal VOC mAP evaluation module at the moment, so you'll have to use the official Matlab evaluation script that comes with the VOCdevkit. Note that the mAP computation formula differs for VOC2007 and VOC2012, and the VOCdevkit for each of these years comes with its own evaluation script that you should use for the respective dataset.